In [ ]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import pickle
import networkx as nx
import numpy as np
import random
import time
import statistics
from itertools import chain 

In [ ]:
# Для получения токена открыть ссылку в браузере:
# Для контроля над своим токеном, нужно изменить client_id на id своего приложение ВК
# https://oauth.vk.com/authorize?client_id=8009498&display=page&scope=friends&response_type=token&v=5.92&state=123456 

group_api_url = 'https://api.vk.com/method/groups.getMembers?group_id='
id_api_url = 'https://api.vk.com/method/friends.get?user_id='
fields = '&fields=id'
count = '&count=100'
offset = '&offset='
v = '&v=5.131'
access_token = '&access_token=TOKEN'

In [ ]:
# Id группы ВК
vk_group_id = 75632872

In [ ]:
#Обращение к vk_api
def extract_members(group_id):

    list_of_members = []
    
    for offset_number in range(0, 3000, 1000):
        
        url = group_api_url + str(group_id) + offset + str(offset_number) + count + fields + access_token + v
        json_response = requests.get(url).json()
        users = json_response['response']['items']
        list_of_members += users
        
    return list_of_members

In [ ]:
#Получение количества подписчиков в сообществе
def get_members_count(group_id):

    url = group_api_url + str(group_id) + access_token + v
    json_response = requests.get(url).json()
    count = json_response['response']['count']
        
    return count

In [ ]:
# Получаем подробную информацию о пользователях
group_members = extract_members(vk_group_id)
print(group_members)

[{'id': 5523, 'first_name': 'Андрей', 'last_name': 'Самсонов', 'can_access_closed': True, 'is_closed': False}, {'id': 32168, 'first_name': 'Марина', 'last_name': 'Жезлова', 'can_access_closed': True, 'is_closed': False}, {'id': 42203, 'first_name': 'Валерия', 'last_name': 'Митропольская', 'can_access_closed': True, 'is_closed': False}, {'id': 80026, 'first_name': 'Ольга', 'last_name': 'Гаранина', 'can_access_closed': False, 'is_closed': True}, {'id': 83851, 'first_name': 'Валерия', 'last_name': 'Цветаева', 'can_access_closed': True, 'is_closed': False}, {'id': 85637, 'first_name': 'Антонина', 'last_name': 'Чижова', 'can_access_closed': True, 'is_closed': False}, {'id': 111476, 'first_name': 'Наталья', 'last_name': 'Гразион', 'can_access_closed': True, 'is_closed': False}, {'id': 133692, 'first_name': 'Наталья', 'last_name': 'Комлева', 'can_access_closed': True, 'is_closed': False}, {'id': 176757, 'first_name': 'Маша', 'last_name': 'Бендер', 'can_access_closed': True, 'is_closed': False

In [ ]:
#Количество подписчиков в сообществе
print(get_members_count(vk_group_id))

8027


In [ ]:
#Полученные подписчики
print(len(group_members))

295


In [ ]:
#Пример данных о подписчиках
group_members[len(group_members)-1]

{'id': 98637329,
 'first_name': 'Руана',
 'last_name': 'Фдс',
 'can_access_closed': True,
 'is_closed': False}

In [ ]:
output = open('group_members_' + str(vk_group_id) + '.pkl', 'wb')
pickle.dump(group_members, output)
output.close()

In [ ]:
with open('group_members_' + str(vk_group_id) + '.pkl', 'rb') as f:
    group_members = pickle.load(f)

In [ ]:
#Список id подписчиков сообщества
members_ids = []
for member in group_members:
    members_ids.append(member['id'])

In [ ]:
#Получение списка друзей, которые также подписаны на данное сообщество
def user_friends_list(user_id):
    
    url = id_api_url + str(user_id) + access_token + v
    
    time.sleep(0.12)
    
    try:
        json_response = requests.get(url).json()
    except requests.exceptions.RequestException:
        print('error')
        return []
    
    if 'error' in json_response.keys():
        print('error')
        return []
    
    friends_inside_community_list = list(set(json_response['response']['items']).intersection(members_ids))
    print('success')
    return friends_inside_community_list

In [ ]:
url = id_api_url + str(154101110) + access_token + v
print(requests.get(url).json())

{'response': {'count': 251, 'items': [17885170, 24681239, 28933061, 44063304, 47507719, 56065716, 57728149, 62081527, 66221313, 68392475, 80496289, 87528713, 91707637, 92341144, 94655253, 98287610, 100855207, 106039618, 106326868, 120196424, 123840463, 127449269, 133831289, 135332203, 139137197, 141119795, 147404018, 149180766, 150503721, 152357494, 152459698, 153410245, 155159728, 156447193, 160437799, 163265521, 163994800, 164495195, 165114276, 165956440, 168001232, 168334930, 169544787, 170438356, 171392101, 171750244, 172516290, 175480408, 176152711, 176305524, 177947983, 181834713, 182397530, 186079332, 186661957, 187202061, 188855526, 188857494, 193607817, 195901751, 195902730, 199576767, 202359872, 202649224, 205024039, 206216437, 208999870, 209202622, 209475114, 213787267, 214606144, 215346577, 217050007, 219294559, 219988044, 224594718, 225185068, 226767051, 229573619, 230346278, 238613781, 239307956, 242355399, 243119152, 243911283, 246886476, 247055854, 252972069, 254285721,

In [ ]:
# Создаем файл, в нем список ребер: пары вершина - вершина 

f1 = open('friends_inside_' + str(vk_group_id) + '.txt', 'w')

for member_id in members_ids:
    
    a = user_friends_list(member_id)
    
    if len(a) != 0:
        
        for friend_id in a:
            f1.write('%d' % member_id)
            f1.write(' ')
            f1.write('%d' % friend_id)
            f1.write('\n')

f1.close()

success
success
success
error
success
success
success
success
success
success
success
success
success
success
success
success
success
error
success
success
error
error
success
success
success
success
error
success
success
success
success
success
success
error
success
success
error
success
success
success
error
success
success
success
success
success
success
success
error
error
success
success
success
success
error
success
error
error
error
success
success
success
error
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
error
error
error
success
success
error
success
success
success
success
error
success
success
success
error
success
success
error
success
success
error
error
success
success
success
success
success
success
error
success
success
success
error
success
error
success
success
success
success
success
success
success
success
success
success
success
success
success
success
error


In [ ]:
# Открываем наш граф в networkx
G1 = nx.read_edgelist('friends_inside_' + str(vk_group_id) + '.txt')

In [ ]:
# Основные показатели графа
print('Число вершин:', len(list(G1.nodes())))
print('Число ребер:', len(list(G1.edges())))

Число вершин: 2959
Число ребер: 24259


In [ ]:
k_cores_size = []
for i in range(1, 25):
    k_cores_size.append(len(nx.k_core(G1, i).nodes()))

In [ ]:
#количество подписчиков, у которых по одному, по два, по три.... друга
for i, n in enumerate(k_cores_size):
    print('Количество вершин в %s-core:' % (i+1), n)

Количество вершин в 1-core: 2959
Количество вершин в 2-core: 2046
Количество вершин в 3-core: 1669
Количество вершин в 4-core: 1452
Количество вершин в 5-core: 1287
Количество вершин в 6-core: 1170
Количество вершин в 7-core: 1083
Количество вершин в 8-core: 981
Количество вершин в 9-core: 938
Количество вершин в 10-core: 893
Количество вершин в 11-core: 828
Количество вершин в 12-core: 773
Количество вершин в 13-core: 722
Количество вершин в 14-core: 658
Количество вершин в 15-core: 613
Количество вершин в 16-core: 573
Количество вершин в 17-core: 539
Количество вершин в 18-core: 517
Количество вершин в 19-core: 494
Количество вершин в 20-core: 471
Количество вершин в 21-core: 444
Количество вершин в 22-core: 420
Количество вершин в 23-core: 396
Количество вершин в 24-core: 382
